Basic example where we express the constraints for a neural net consisting of a single convolution layer.

In [1]:
include("nn_ops.jl");

In [7]:
# We specify the parameters for the size of the problem that we are solving.
batch = 1
in_height = 6
in_width = 6
stride_height = 2
stride_width = 2
in_channels = 1
filter_height = 2
filter_width = 2
out_channels = 4
bigM = 10000

pooled_height = round(Int, in_height/stride_height, RoundUp)
pooled_width = round(Int, in_width/stride_width, RoundUp)

# Fix random seed so that we get consistent results
srand(5)

# We select a random filter for the convolution.
filter = rand(-10:10, filter_height, filter_width, in_channels, out_channels)

# We select some random input, and determine the activations at each layer
# for that input.
# `x_conv_relu_maxpool_actual` is the target output that we will seek to
# achieve by perturbing `x_perturbed`.
x_actual = rand(-10:10, batch, in_height, in_width, in_channels)
x_conv_relu_maxpool_actual = NNOps.convlayer(x_actual, filter, (1, 2, 2, 1));

x_perturbed = rand(-10:10, batch, in_height, in_width, in_channels);

In [8]:
using JuMP
using Gurobi

m = Model(solver=GurobiSolver())

# `ve` is the tensor of perturbations, while `vx` is the tensor representing the
# original input which we are perturbing.
# Our objective in this optimization is to minimize the l-2 norm of the
# perturbations.
@variable(m, ve[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@objective(m, Min, sum(ve.^2))
@variable(m, vx[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@constraint(m, vx .== x_perturbed)

vx_conv_relu_maxpool = NNOps.convlayerconstraint(m, vx+ve, filter, (2, 2), 10000)
@constraint(m, vx_conv_relu_maxpool .== x_conv_relu_maxpool_actual);

In [9]:
status = solve(m)

println("\nOriginal Input:\n", x_perturbed)
println("\nPerturbation:\n", getvalue(ve))

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Optimize a model with 684 rows, 468 columns and 2236 nonzeros
Model has 36 quadratic objective terms
Variable types: 288 continuous, 180 integer (180 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 440 rows and 234 columns
Presolve time: 0.02s
Presolved: 244 rows, 234 columns, 800 nonzeros
Presolved model has 34 quadratic objective terms
Variable types: 136 continuous, 98 integer (98 binary)

Root relaxation: objective 1.161818e+03, 299 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1161.81844    0   44          - 1161.81844      -     -    0s
     0     0 1161.81921    0   45          - 1161.81921   

In [12]:
m

Minimization problem with:
 * 684 linear constraints
 * 468 variables: 180 binary
Solver is Gurobi

In [13]:
filter

2×2×1×4 Array{Int64,4}:
[:, :, 1, 1] =
 -4  -2
 -3   0

[:, :, 1, 2] =
  5  -3
 -7   0

[:, :, 1, 3] =
  1   2
 10  -7

[:, :, 1, 4] =
  2   9
 -5  -5

In [6]:
# Sanity checks comparing the value of the activations from feeding
# the perturbed input to the net to the value of the activations from
# feeding the actual input.

x_conv_perturbed = NNOps.conv2d(x_perturbed+getvalue(ve), filter);
x_conv_relu_perturbed = NNOps.relu(x_conv_perturbed);
x_conv_relu_maxpool_perturbed = NNOps.maxpool(x_conv_relu_perturbed, (1, 2, 2, 1));

x_conv_actual = NNOps.conv2d(x_actual, filter);
x_conv_relu_actual = NNOps.relu(x_conv_actual);


In [7]:
conv_delta = maximum(abs(x_conv_perturbed - x_conv_actual))
conv_relu_delta = maximum(abs(x_conv_relu_perturbed - x_conv_relu_actual))
conv_relu_maxpool_delta = maximum(abs(x_conv_relu_maxpool_perturbed - x_conv_relu_maxpool_actual))
println("Maximum difference between convolution activations: ", conv_delta)
println("Maximum difference between relu activations: ", conv_relu_delta)
println("Maximum difference between maxpool activations: ", conv_relu_maxpool_delta)

Maximum difference between convolution activations: 506.34253876129304
Maximum difference between relu activations: 295.0
Maximum difference between maxpool activations: 3.473985543678282e-10
